### 언더샘플링_임계값 조정

In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.svm as svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics 
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import cross_val_score, cross_validate
from lightgbm import LGBMClassifier
import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier

#추가
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, fbeta_score


In [116]:
def get_clf_eval(y_test, y_pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test,pred_proba)
    
# 아래 추가함 (pr_score, f2 score, gmean)
# pr_score : 모델이 양성 클래스를 얼마나 정확하게 찾아내는지에 대한 정보를 제공(클래스 불균형이 존재할 때 유용)
# fbeta_score : 정밀도(Precision)와 재현율(Recall)의 조화 평균을 계산하는 지표.재현율에 더 큰 가중치를 두는데, 이는 False Negatives를 최소화하는 데 중점
# G-Mean : Sensitivity와 Specificity의 조화 평균. 클래스 간의 불균형을 고려하여 모델의 성능을 측정하며, 특히 Positive 클래스의 예측 성능이 중요한 경우에 유용
    confusion = confusion_matrix(y_test, y_pred)
    pr_score = average_precision_score(y_test, y_pred)
    f2 = fbeta_score(y_test, y_pred, beta=2)

    # G-mean 계산
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    tpr = tp / (tp + fn)  # True Positive Rate
    tnr = tn / (tn + fp)  # True Negative Rate
    gmean = np.sqrt(tpr * tnr)

    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))
    print('Fbeta :{:.4f}'.format(f2))
    print('평균 정밀도 : {:.4f}'.format(pr_score))
    print('gmean : {:.4f}'.format(gmean))

In [117]:
# dataset 작업요
df = pd.read_csv('./final_datasets.csv', index_col=0)
df.head(3)

,TARGET,자녀 수,가족 구성원 수,나이,가입연수,월간 수입,성별,차량 소유 여부,부동산 소유 여부,수입 유형,...,도시구분,home_shape,car_home,가족크기범주,combinedFY,cons_low,cons_lab,bus_low,bus_drv,bus_lab
0,0,2,4.0,39,23.0,1504500.0,1,1,1,3,...,1,7,3,1,0,0,0,0,0,0
1,0,0,2.0,45,16.0,4956000.0,0,1,0,1,...,2,3,2,0,0,0,0,0,0,0
2,0,0,2.0,32,9.0,2478000.0,1,0,1,1,...,2,7,1,0,0,0,0,0,0,0


In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df['TARGET'].value_counts()

In [118]:
# feature와 label 분리
feature = df.drop('TARGET', axis=1)
label = df['TARGET']

In [119]:
# train test 분리
X_train_old, X_test, y_train_old , y_test = train_test_split(feature, label, test_size=0.3 , random_state=42)

In [120]:
y_train_old.value_counts()

TARGET
0    37484
1     4507
Name: count, dtype: int64

In [121]:
# 언더샘플링 - 소수라벨과 다수라벨 비율 1:1

from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy='majority',random_state=42)
X_train, y_train = undersample.fit_resample(X_train_old, y_train_old)

In [ ]:
# 오버샘플링 - 소수라벨과 다수라벨 비율 1:1 / 오버 샘플링이 좋음 투터님 피셜

from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')
X_train, y_train = oversample.fit_resample(X_train_old, y_train_old)

In [ ]:
# # SMOTE

# from imblearn.over_sampling import SMOTE
# smote_sample = SMOTE(sampling_strategy='minority') 
# X_train, y_train = smote_sample.fit_resample(X_train_old, y_train_old)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# 클래스 불균형 계산
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

# 클래스 가중치를 딕셔너리로 변환
class_weight_dict = {cls: weight for cls, weight in zip(np.unique(y_train), class_weights)}
# # 타겟 클래스에 대한 가중치 설정
# class_weights = {0: 1, 1: 1.3}  # 예시로 클래스 1에 더 큰 가중치를 줌

# # RandomForestClassifier 모델 생성 시 class_weight 파라미터 설정
# model = RandomForestClassifier(class_weight=class_weights)

# # 모델 학습
# model.fit(X_train, y_train)

# # 테스트 데이터로 예측
# y_pred = model.predict(X_test)

# # 혼동 행렬 및 분류 보고서 출력
# print("Confusion Matrix:")
# print(confusion_matrix(y_test, y_pred))

# print("\nClassification Report:")
# print(classification_report(y_test, y_pred))

In [ ]:
y_train.value_counts()

In [122]:
# 1. Decision Tree를 이용한 학습

dt = DecisionTreeClassifier(random_state=43)

# 교차검증을 통한 과적합 or 과소적합 여부 확인
cross_val = cross_val_score(dt , X_train , y=y_train ,cv=5, scoring='accuracy')
print(np.mean(cross_val))

0.514421641572838


In [123]:
# GridSearchCV를 적용해 Decision Tree의 교차검증 및 하이퍼파라미터 튜닝

parameters = {'max_depth' : [2, 3, 4, 5],
             'min_samples_split' : [1, 3, 5, 7, 9]}

grid_dt = GridSearchCV(dt, param_grid = parameters, cv=3, refit=True)
grid_dt.fit(X_train, y_train)

dt = grid_dt.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_dt.best_params_}")
print(f"최고 예측 정확도: {grid_dt.best_score_:.4f}")

최적 하이퍼 파라미터: {'max_depth': 4, 'min_samples_split': 3}
최고 예측 정확도: 0.5924


In [133]:
# 임곗값 조정을 통한 recall값 향상
dt.fit(X_train, y_train)
pred = dt.predict(X_test)
pred_proba = dt.predict_proba(X_test)[:, 1].reshape(-1, 1)

binarizer = Binarizer(threshold=0.4).fit(pred_proba)
custom_pred = binarizer.transform(pred_proba)

get_clf_eval(y_test, custom_pred, pred_proba)
print(confusion_matrix(y_test, pred))

오차행렬:
 [[ 4706 11371]
 [  337  1583]]

정확도: 0.3494
정밀도: 0.1222
재현율: 0.8245
F1: 0.2129
AUC: 0.6145
Fbeta :0.3836
평균 정밀도 : 0.1195
gmean : 0.4913
[[10477  5600]
 [  888  1032]]


In [134]:
# 트리 기반 모델의 특성 중요도 확인
for feature, importance in zip(X_train.columns, dt.feature_importances_):
    print(f"{feature}: {importance}")
# 0에 가까울수록 중요도 낮고, 0.4나 0.5 정도가 중요도 높다할 수 있음

자녀 수: 0.0
가족 구성원 수: 0.0
나이: 0.2947634142948027
가입연수: 0.08610338123023548
월간 수입: 0.039962578223135516
성별: 0.1622770809725894
차량 소유 여부: 0.0
부동산 소유 여부: 0.0
수입 유형: 0.0
최종 학력: 0.28471456014147767
결혼 여부: 0.0
주거 형태: 0.0
휴대전화 소유 여부: 0.0
이메일 소유 여부: 0.0
직업: 0.03274418528028609
산업군: 0.0
도시구분: 0.0
home_shape: 0.027053726818775672
car_home: 0.07238107303869758
가족크기범주: 0.0
combinedFY: 0.0
cons_low: 0.0
cons_lab: 0.0
bus_low: 0.0
bus_drv: 0.0
bus_lab: 0.0


In [135]:
# 2. RandomForest를 이용한 학습

rf = RandomForestClassifier(random_state=43)

cross_val = cross_val_score(rf , X_train , y=y_train ,cv=5, scoring='accuracy')
print(np.mean(cross_val))


0.5680047374489305


In [136]:
# 하이퍼파라미터 조정

parameters = {
    'max_depth' : [6, 8, 12],
    'min_samples_split' : [16, 24]
}

grid_rf = GridSearchCV(rf, param_grid = parameters, cv=3, refit=True)
grid_rf.fit(X_train, y_train)

rf = grid_rf.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_rf.best_params_}")
print(f"최고 예측 정확도: {grid_rf.best_score_:.4f}")

최적 하이퍼 파라미터: {'max_depth': 6, 'min_samples_split': 16}
최고 예측 정확도: 0.5988


In [137]:
# 임계값 조정 및 적용

rf.fit(X_train, y_train)
pred = rf.predict(X_test)
pred_proba = rf.predict_proba(X_test)[:, 1].reshape(-1, 1)

binarizer = Binarizer(threshold=0.45).fit(pred_proba)
custom_pred = binarizer.transform(pred_proba)

get_clf_eval(y_test, custom_pred, pred_proba)


오차행렬:
 [[6222 9855]
 [ 435 1485]]

정확도: 0.4282
정밀도: 0.1310
재현율: 0.7734
F1: 0.2240
AUC: 0.6352
Fbeta :0.3904
평균 정밀도 : 0.1255
gmean : 0.5471


In [ ]:
# 트리 기반 모델의 특성 중요도 확인
for feature, importance in zip(X_train.columns, rf.feature_importances_):
    print(f"{feature}: {importance}")

In [ ]:
# 3. Logistic regrssion 을 이용한 학습

lr = LogisticRegression(class_weight=class_weight_dict, random_state=43)

cross_val = cross_val_score(lr, X_train , y=y_train ,cv=5, scoring='accuracy')
print(np.mean(cross_val))


In [ ]:
parameters = {'penalty': ['l2','l1'],
          'C':[0.01,0.1,1,10]}

grid_lr = GridSearchCV(lr, param_grid = parameters, cv=3, refit=True)
grid_lr.fit(X_train, y_train)

lr = grid_lr.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_lr.best_params_}")
print(f"최고 예측 정확도: {grid_lr.best_score_:.4f}")

In [ ]:
# 임계값 조정 및 모델 적용

lr.fit(X_train, y_train)
pred = lr.predict(X_test)
pred_proba = lr.predict_proba(X_test)[:, 1].reshape(-1, 1)

binarizer = Binarizer(threshold=0.47).fit(pred_proba)
custom_pred = binarizer.transform(pred_proba)

get_clf_eval(y_test , custom_pred, pred_proba)

In [ ]:
# 학습된 모델의 특성 중요도 확인
feature_importance = lr.coef_[0]  # 특성의 가중치 또는 중요도

# 특성별 중요도 출력
for feature, importance in zip(X_train.columns, feature_importance):
    print(f"{feature}: {importance}")

# 가중치의 절댓값이 클수록 중요도 높음
# 0에 가까울수록 중요도 낮음

In [ ]:
# 4. KNN 을 이용한 학습

knn = KNeighborsClassifier()
# knn에는 무작위성 추출(random_state)를 하지 않음.

cross_val = cross_val_score(knn , X_train , y=y_train ,cv=5, scoring='accuracy')
print(np.mean(cross_val))


In [ ]:
# 하이퍼파라미터 조정

parameters = {'n_neighbors': [3, 5, 7, 9],
              'weights': ['uniform', 'distance']
          }

grid_knn = GridSearchCV(knn, param_grid = parameters, cv=3, refit=True)
grid_knn.fit(X_train, y_train)

knn = grid_knn.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_knn.best_params_}")
print(f"최고 예측 정확도: {grid_knn.best_score_:.4f}")

In [ ]:
# 임계값 조정 및 모델 적용

knn.fit(X_train, y_train)
pred = knn.predict(X_test)
pred_proba = knn.predict_proba(X_test)[:, 1].reshape(-1, 1)

binarizer = Binarizer(threshold=0.45).fit(pred_proba)
custom_pred = binarizer.transform(pred_proba)

get_clf_eval(y_test , custom_pred, pred_proba)

In [ ]:
# 5. Lightgbm을 이용한 학습

lgbm= LGBMClassifier(class_weight=class_weight_dict,random_state=43)

cross_val = cross_val_score(lgbm, X_train , y=y_train ,cv=5, scoring='accuracy')
print(np.mean(cross_val))

In [ ]:
# 하이퍼파라미터 조정

parameters = {'n_neighbors': [100, 500],
              'learning_rate': [0.05, 0.1]
          }

grid_lgbm = GridSearchCV(lgbm, param_grid = parameters, cv=2, verbose=1, refit=True)
grid_lgbm.fit(X_train, y_train)

lgbm = grid_lgbm.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_lgbm.best_params_}")
print(f"최고 예측 정확도: {grid_lgbm.best_score_:.4f}")

In [ ]:
# 임계값 조정 및 모델 적용

lgbm.fit(X_train, y_train)
pred = lgbm.predict(X_test)
pred_proba = lgbm.predict_proba(X_test)[:, 1].reshape(-1, 1)

binarizer = Binarizer(threshold=0.45).fit(pred_proba)
custom_pred = binarizer.transform(pred_proba)

get_clf_eval(y_test , custom_pred, pred_proba)


In [ ]:
# 6. XGboost 
xgb = XGBClassifier(class_weight=class_weight_dict, random_state=43)

cross_val = cross_val_score(xgb, X_train, y=y_train, cv=5, scoring='accuracy')
print(np.mean(cross_val))

In [ ]:
# 하이퍼파라미터 조정
parameters = {'learning_rat': [0.01, 0.1],  
              'n_estimators': [50, 100, 200]
          }

grid_xgb = GridSearchCV(xgb, param_grid=parameters, cv=5, verbose=1, refit=True)
grid_xgb.fit(X_train, y_train)

xgb = grid_xgb.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_xgb.best_params_}")
print(f"최고 예측 정확도: {grid_xgb.best_score_:.4f}")

In [ ]:
# 임계값 조정 및 모델 적용

xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)
pred_proba = xgb.predict_proba(X_test)[:, 1].reshape(-1, 1)

binarizer = Binarizer(threshold=0.45).fit(pred_proba)
custom_pred = binarizer.transform(pred_proba)

get_clf_eval(y_test , custom_pred, pred_proba)

In [ ]:
# 한 클래스의 샘플 수가 다른 클래스에 비해 월등히 많은 경우, 
# 모델이 많은 샘플이 있는 클래스에 더 치우쳐 학습